## Practical 9
### Implement Naive Bayes classifier

In [32]:
# https://www.kaggle.com/springboardroger/naive-bayes-name-gender-classifier

In [1]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import MultinomialNB
import requests
from bs4 import BeautifulSoup
import matplotlib as mlp
mlp.use("TKAgg")
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
mlp.rcParams.update({'font.family': "Open Sans", 'font.size' : 16})

In [3]:
import pandas as pd
df = pd.read_csv("yob2020.txt")
# df = pd.read_csv("NBC-data.csv")
df.head()

,name,gender,count
0,Olivia,F,17535
1,Emma,F,15581
2,Ava,F,13084
3,Charlotte,F,13003
4,Sophia,F,12976


In [4]:
df.tail()

,name,gender,count
31266,Zykell,M,5
31267,Zylus,M,5
31268,Zymari,M,5
31269,Zyn,M,5
31270,Zyran,M,5


In [5]:
namechart = df.groupby(['name', 'gender'], as_index = False)['count'].sum()
namechart.head(5)

,name,gender,count
0,Aabha,F,5
1,Aabriella,F,7
2,Aadam,M,15
3,Aadarsh,M,9
4,Aadav,M,7


In [7]:
namechartdiff = namechart.reset_index().pivot('name', 'gender', 'count')
namechartdiff = namechartdiff.fillna(0)
namechartdiff["Mpercent"] = ((namechartdiff["M"] - namechartdiff["F"])/(namechartdiff["M"] + namechartdiff["F"]))
namechartdiff['gender'] = np.where(namechartdiff['Mpercent'] > 0.001, 'male', 'female')
namechartdiff.head()

gender,F,M,Mpercent,gender
name,,,,
Aabha,5.0,0.0,-1.0,female
Aabriella,7.0,0.0,-1.0,female
Aadam,0.0,15.0,1.0,male
Aadarsh,0.0,9.0,1.0,male
Aadav,0.0,7.0,1.0,male


In [8]:
namechartdiff.gender.value_counts()

female    15903
male      12710
Name: gender, dtype: int64

In [9]:
char_vectorizer = CountVectorizer(analyzer='char', ngram_range=(2, 2))
X = char_vectorizer.fit_transform(namechartdiff.index)
X = X.tocsc()
y = (namechartdiff.gender == 'male').values.astype(np.int)
print(X)

  (0, 0)	1
  (1, 0)	1
  (2, 0)	1
  (3, 0)	1
  (4, 0)	1
  (5, 0)	1
  (6, 0)	1
  (7, 0)	1
  (8, 0)	1
  (9, 0)	1
  (10, 0)	1
  (11, 0)	1
  (12, 0)	1
  (13, 0)	1
  (14, 0)	1
  (15, 0)	1
  (16, 0)	1
  (17, 0)	1
  (18, 0)	1
  (19, 0)	1
  (20, 0)	1
  (21, 0)	1
  (22, 0)	1
  (23, 0)	1
  (24, 0)	1
  :	:
  (11733, 564)	1
  (12849, 564)	1
  (12850, 564)	1
  (12851, 564)	1
  (12852, 564)	1
  (12853, 564)	1
  (12854, 564)	1
  (12855, 564)	1
  (12856, 564)	1
  (15922, 564)	1
  (17426, 564)	1
  (17427, 564)	1
  (17428, 564)	1
  (17429, 564)	1
  (19127, 564)	1
  (19128, 564)	1
  (19424, 564)	1
  (19918, 564)	1
  (19919, 564)	1
  (19943, 564)	1
  (20002, 564)	1
  (21548, 564)	1
  (21549, 564)	1
  (26516, 564)	1
  (26517, 564)	1


In [10]:
itrain, itest = train_test_split(range(namechartdiff.shape[0]), train_size=0.7)
mask=np.ones(namechartdiff.shape[0], dtype='int')
mask[itrain]=1
mask[itest]=0
mask = (mask==1)

In [11]:
Xtrainthis=X[mask]
Ytrainthis=y[mask]
Xtestthis=X[~mask]
Ytestthis=y[~mask]
clf = MultinomialNB(alpha = 1)
clf.fit(Xtrainthis, Ytrainthis)
training_accuracy = clf.score(Xtrainthis,Ytrainthis)
test_accuracy = clf.score(Xtestthis,Ytestthis)
        
print(training_accuracy)
print(test_accuracy)

0.7386289879674472
0.7169151910531221


In [12]:
def lookup(x):
    str(x)
    new = char_vectorizer.transform([x])
    y_pred = clf.predict(new)
    if (y_pred == 1):
        print("This is most likely a male name!")
    else:
        print("This is most likely a female name!")

In [13]:
lookup("Roger")

This is most likely a male name!


In [14]:
lookup("Ramesh")

This is most likely a male name!


In [15]:
lookup("manisha")

This is most likely a female name!


In [16]:
lookup("siddhi")

This is most likely a male name!


In [17]:
lookup("aakash")

This is most likely a male name!
